Notebook to predict N+1 command where N is 1..4 and command sets can be up to len 5

Construct Graph Function

In [1]:
from graph import Node

def construct_graph(command_list, command_dict={}):

    filter_empty = lambda x: (len(x) > 0)
    cur_node = None
    child_node = None
    
    for session in command_list:
        # remove commands of length 0 ( not good to modify list within loop )
        
        session = list(filter(filter_empty, session))

        try: 
            first_cmd = session[0]
        except Exception as inst:
            print(session)
            continue

        program = session[0].split()[0]

        if command_dict.get(program) is None:
            cur_node = Node(program=program, frequency=1)
            command_dict[program] = cur_node 
        else:
            cur_node = command_dict.get(program)
            cur_node.frequency += 1

        if cur_node.commands.get(first_cmd) is None:
            cur_node.commands[first_cmd] = 1
        else:
            cur_node.commands[first_cmd] += 1

        for cmd in range(1, len(session)):

            if session[cmd].isprintable() is False:
                break
            else:
                program = session[cmd].split()[0]

            if cur_node.children.get(program) is None:
                child_node = Node(program = program, frequency=1)
                cur_node.children[program] = child_node
            else:
                child_node = cur_node.children.get(program)
                child_node.frequency += 1
            
            if child_node.commands.get(session[cmd]) is None:
                child_node.commands[session[cmd]] = 1
            else:
                child_node.commands[session[cmd]] += 1
            
            cur_node = child_node
        
                
        
    return command_dict


Get Prediction Function

In [2]:
def get_prediction(command_list: list[str], graph, result_size=5, graph_depth=4):
    if len(command_list) == 0:
        return None

    commands = command_list[:graph_depth]
    # should be labeled last command
    last_command = command_list[graph_depth]

    if len(commands) == 0:
        return None, [-1]

    program = commands[0].split()[0]

    frequencies = []
    
    if graph.get(program) is None:
        frequencies.append(-1)
        return None, frequencies
    else:
        node = graph[program]
        frequencies.append(node.frequency)

    for command in commands[1:]:
        program = command.split()[0]
        if node.children.get(program) is not None:
            node = node.children[program]
            frequencies.append(node.frequency)
        else:
            frequencies.append(-1)
            return None, frequencies
    
    # last frequency if it has it
    next_program = last_command.split()[0]

    if node.children.get(next_program) is not None:
        frequencies.append(node.children[next_program].frequency)
    else:
        frequencies.append(-1)
    
    return node.get_prediction(previous_command=last_command, num_to_return=result_size), frequencies

Accuracy Function

In [3]:
from thefuzz import fuzz
from thefuzz import process
from sklearn.model_selection import train_test_split

import pprint
pp = pprint.PrettyPrinter(indent=4)

def append_list(lst, results, commands, graph_depth):
    lst.append({"Results": results, "Expected": commands[graph_depth], "Command Sequence": commands})

def get_accuracy(command_subsets, fail_list = [], succeed_list = [], correct_5_not_3= [], correct_15_not_5 = [], print_fails=False, graph_depth=4):
    train_data, test_data = train_test_split(command_subsets, test_size=0.2, train_size=0.8, random_state=42)

    train_graph = construct_graph(train_data)

    # filter out nodes that can't get prediction because not enough length so don't contribute and will cause error
    test_data = [x for x in test_data if len(x) > graph_depth]
    test_size = len(test_data)

    return_3 = 3
    return_5 = 5
    return_15 = 15

    correct = 0
    correct_with_5 = 0
    correct_with_15 = 0
    first_prediction = 0
    has_prediction = 0
    incorrect = 0
    none_count = 0

    for commands in test_data:
        results_3, frequencies = get_prediction(commands, train_graph, return_3, graph_depth)
        results_5, _ = get_prediction(commands, train_graph, return_5, graph_depth)
        results_15, _ = get_prediction(commands, train_graph, return_15, graph_depth)

        command_freq = [(commands[i], frequencies[i]) for i in range(len(frequencies))]
        
        prev_correct = correct

        if results_3 is not None:
            has_prediction += 1

            for i in range(len(results_3)):
                if fuzz.ratio(results_3[i][0], commands[graph_depth]) > 85:
                    correct += 1

                    if correct <= 100:
                        append_list(succeed_list, results_5, command_freq, graph_depth)

                    if i == 0:
                        first_prediction += 1
                
                break

            if prev_correct == correct:
                incorrect += 1

            for i in range(len(results_5)):
                if fuzz.ratio(results_5[i][0], commands[graph_depth]) > 85:
                    correct_with_5 += 1

                    if correct_with_5 - correct <= 100:
                        append_list(correct_5_not_3, results_5, command_freq, graph_depth)

                    break

            for i in range(len(results_15)):
                if fuzz.ratio(results_15[i][0], commands[graph_depth]) > 85:
                    correct_with_15 += 1

                    if correct_with_15 - correct <= 100:
                        append_list(correct_15_not_5, results_15, command_freq, graph_depth)

                    break

        else:
            none_count += 1

        if results_5 is not None and prev_correct == correct and print_fails:
            append_list(fail_list, results_5, command_freq, graph_depth)  
                      
    return 'Correct Proportion: {:.2f}% |\n Correct in 5 not 3: {:.2f}% |\n Correct in 15 not 5: {:.2f}% |\n Has Prediction and is Correct: {:.2f}% |\n Incorrect Proportion: {:.2f}% |\n None Proportion: {:.2f}% |\n First Prediction: {:.2f}%'.format(100 * correct/test_size, 100 * correct_with_5/test_size, 100 * correct_with_15/test_size, 100 * correct/has_prediction, 100 * incorrect/test_size, 100 * none_count/test_size, 100 * first_prediction/test_size)

In [4]:
from parse import Parser

parser = Parser()

100%|██████████| 56/56 [00:08<00:00,  6.85it/s]


All Commands Parse and Replace Args

In [5]:
subset_size = 5

science_session = parser.parse_commands_per_session(parser.scientists_files)
science_session = parser.parse_commands_into_subsets_sliding_window(science_session, subset_size)
science_session = parser.replace_args_nested(science_session)

experienced_session = parser.parse_commands_per_session(parser.experienced_files)
experienced_session = parser.parse_commands_into_subsets_sliding_window(experienced_session, subset_size)
experienced_session = parser.replace_args_nested(experienced_session)

non_session = parser.parse_commands_per_session(parser.non_programmers_files)
non_session = parser.parse_commands_into_subsets_sliding_window(non_session, subset_size)
non_session = parser.replace_args_nested(non_session)

novice_session = parser.parse_commands_per_session(parser.novice_files)
novice_session = parser.parse_commands_into_subsets_sliding_window(novice_session, subset_size)
novice_session = parser.replace_args_nested(novice_session)

100%|██████████| 56/56 [00:08<00:00,  6.25it/s]


Next 2 cells for testing

In [5]:
from random import sample
import math

science_session_sample = parser.parse_commands_per_session(sample(parser.scientists_files, math.floor(0.2 * len(parser.scientists_files))))
science_session_sample = parser.parse_commands_into_subsets_sliding_window(science_session_sample, 5)
science_session_sample = parser.replace_args_nested(science_session_sample)

100%|██████████| 10/10 [00:02<00:00,  4.93it/s]


In [25]:
fail_list = []
succeed_list = []
print("Scientist session:\n", get_accuracy(science_session_sample, fail_list=fail_list, print_fails=True, succeed_list=succeed_list, graph_depth=4))

"""
Scientist session:
 Correct Proportion: 17.93% |
 Correct in 15 not 3: 18.90% |
 Has Prediction and is Correct: 20.37% |
 Incorrect Proportion: 70.09% |
 None Proportion: 11.99% |
 First Prediction: 12.92%
"""

Scientist session:
 Correct Proportion: 76.47% |
 Correct in 5 not 3: 82.94% |
 Correct in 15 not 5: 83.98% |
 Has Prediction and is Correct: 76.85% |
 Incorrect Proportion: 23.04% |
 None Proportion: 0.50% |
 First Prediction: 76.47%


'\nScientist session:\n Correct Proportion: 17.93% |\n Correct in 15 not 3: 18.90% |\n Has Prediction and is Correct: 20.37% |\n Incorrect Proportion: 70.09% |\n None Proportion: 11.99% |\n First Prediction: 12.92%\n'

In [8]:
succeed_list

[{'Results': [('fred $0', 2300),
   ('spell $0', 106),
   ('fred', 73),
   ('itroff $0 | $1', 38),
   ('cp $0 $0.b', 35)],
  'Expected': ('fred $0', 48),
  'Command Sequence': [('fred $0', 1608),
   ('fred $0', 436),
   ('fred $0', 234),
   ('fred $0', 114),
   ('fred $0', 48)]},
 {'Results': [('fred $0', 100), ('fred', 73), ('ptroff -me -Palw2 $0', 37)],
  'Expected': ('fred $0', 4),
  'Command Sequence': [('ptroff -me -Palw2 $0', 524),
   ('fred $0', 370),
   ('ptroff -me -Palw2 $0', 278),
   ('ptroff -me -Palw2 $0', 18),
   ('fred $0', 4)]},
 {'Results': [('grep $0 $1', 1600),
   ('grep $0 $1.c', 273),
   ('grep $0', 82),
   ('grep $0 $1 $2 $3 $0.c', 65),
   ('grep $0 $1 $2 $3 $1.c', 65)],
  'Expected': ('grep $0 $1', 48),
  'Command Sequence': [('grep $0 $1', 476),
   ('grep $0 $1', 228),
   ('grep $0 $1', 182),
   ('grep $0 $1', 104),
   ('grep $0 $1', 48)]},
 {'Results': [('ps $0', 500)],
  'Expected': ('ps $0', 10),
  'Command Sequence': [('dobis', 558),
   ('ps $0', 106),
   ('

Accuracy predicting N+1 command from N commands with all commands in graph

In [17]:
for i in range(1, 5):
    print("Accuracy predicting command {} from commands {} through {}:\n".format(i+1, 1, i))

    print("Scientist session:\n", get_accuracy(science_session, graph_depth=i))
    print("Experienced session:\n", get_accuracy(experienced_session, graph_depth=i))
    print("Non programmer session:\n", get_accuracy(non_session, graph_depth=i))
    print("Novice session:\n", get_accuracy(novice_session, graph_depth=i))

Accuracy predicting command 2 from commands 1 through 1:

Scientist session:
 Correct Proportion: 65.73% |
 Correct in 15 not 3: 79.35% |
 Has Prediction and is Correct: 65.75% |
 Incorrect Proportion: 34.23% |
 None Proportion: 0.04% |
 First Prediction: 41.89%
Experienced session:
 Correct Proportion: 70.35% |
 Correct in 15 not 3: 83.56% |
 Has Prediction and is Correct: 70.38% |
 Incorrect Proportion: 29.61% |
 None Proportion: 0.04% |
 First Prediction: 45.26%
Non programmer session:
 Correct Proportion: 67.61% |
 Correct in 15 not 3: 79.27% |
 Has Prediction and is Correct: 67.63% |
 Incorrect Proportion: 32.36% |
 None Proportion: 0.03% |
 First Prediction: 42.33%
Novice session:
 Correct Proportion: 78.65% |
 Correct in 15 not 3: 88.79% |
 Has Prediction and is Correct: 78.71% |
 Incorrect Proportion: 21.28% |
 None Proportion: 0.07% |
 First Prediction: 51.53%
Accuracy predicting command 3 from commands 1 through 2:

Scientist session:
 Correct Proportion: 79.59% |
 Correct in

Save files for all data

In [7]:
from tqdm import tqdm
import os
import pprint

pp = pprint.PrettyPrinter(indent=4)

dataset_names = ["Science data", "Experienced data", "Non programmer data", "Novice data"]
datasets = [science_session, experienced_session, non_session, novice_session]

for i in tqdm(range(len(datasets))):
    for j in range (1, 5):
        fail_list = []
        succeed_list = []
        correct_5_not_3 = []
        correct_15_not_5 = []

        get_accuracy(datasets[i], fail_list=fail_list, succeed_list=succeed_list, correct_5_not_3=correct_5_not_3, correct_15_not_5=correct_15_not_5, print_fails=True, graph_depth=j)

        os.makedirs(os.path.dirname("fails/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("fails/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(fail_list))

        os.makedirs(os.path.dirname("successes/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("successes/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(succeed_list))

        os.makedirs(os.path.dirname("correct_5_not_3/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("correct_5_not_3/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(correct_5_not_3))

        os.makedirs(os.path.dirname("correct_15_not_5/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("correct_15_not_5/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(correct_15_not_5))


 50%|█████     | 2/4 [19:14<18:43, 561.84s/it]

In [16]:
i = 3
fail_list = []

print("Scientist session:\n", get_accuracy(science_session, fail_list=fail_list, print_fails=True, succeed_list=succeed_list, graph_depth=i))
print("Experienced session:\n", get_accuracy(experienced_session, graph_depth=i))
print("Non programmer session:\n", get_accuracy(non_session, graph_depth=i))
print("Novice session:\n", get_accuracy(novice_session, graph_depth=i))

Scientist session:
 Correct Proportion: 81.64% |
 Correct in 15 not 3: 86.28% |
 Has Prediction and is Correct: 82.36% |
 Incorrect Proportion: 17.48% |
 None Proportion: 0.88% |
 First Prediction: 64.64%
Experienced session:
 Correct Proportion: 82.49% |
 Correct in 15 not 3: 86.02% |
 Has Prediction and is Correct: 83.36% |
 Incorrect Proportion: 16.46% |
 None Proportion: 1.05% |
 First Prediction: 67.15%
Non programmer session:
 Correct Proportion: 78.38% |
 Correct in 15 not 3: 83.82% |
 Has Prediction and is Correct: 78.85% |
 Incorrect Proportion: 21.02% |
 None Proportion: 0.60% |
 First Prediction: 59.31%
Novice session:
 Correct Proportion: 86.38% |
 Correct in 15 not 3: 90.96% |
 Has Prediction and is Correct: 86.82% |
 Incorrect Proportion: 13.12% |
 None Proportion: 0.50% |
 First Prediction: 65.68%


In [15]:
fail_list

[{'Results': [('ls', 60),
   ('su $0', 60),
   ('cd $0', 60),
   ('chmod $0 $1', 46),
   ('cd $0../TW2', 46)],
  'Expected': ('lpr -Pplsinc $0', -1),
  'Command Sequence': [('ls', 165400),
   ('emacs $0.c', 5810),
   ('cd', 442),
   ('cd $0', 56),
   ('lpr -Pplsinc $0', -1)]},
 {'Results': [('life > $0.out', 96),
   ('mail', 60),
   ('readnews', 21),
   ('grep $0 $1.c $1.h', 1)],
  'Expected': ('limits', -1),
  'Command Sequence': [('readnews', 4856),
   ('fg', 723),
   ('life > $0.out3', 198),
   ('fg', 30),
   ('limits', -1)]},
 {'Results': [],
  'Expected': ('hol', -1),
  'Command Sequence': [('imagen $0 $1', 168),
   ('roff -a2 $0 $1', 3),
   ('e $0.ml', 3),
   ('rm $0.th', 3),
   ('hol', -1)]},
 {'Results': [('rm $0.th', 45)],
  'Expected': ('hol < $0.ml > $0 $1', -1),
  'Command Sequence': [('rm $0.th', 35170),
   ('e', 1922),
   ('lpr $0.ml $1', 22),
   ('rm $0.th', 9),
   ('hol < $0.ml > $0 $1', -1)]},
 {'Results': [],
  'Expected': ('ls', -1),
  'Command Sequence': [('emacs $0

Sampled Commands Parse and Replace Args

In [59]:
from random import sample
import math

subset_size = 5

science_session_sample = parser.parse_commands_per_session(sample(parser.scientists_files, math.floor(0.2 * len(parser.scientists_files))))
science_session_sample = parser.parse_commands_into_subsets_sliding_window(science_session_sample, subset_size)
science_session_sample = parser.replace_args_nested(science_session_sample)

experienced_session_sample = parser.parse_commands_per_session(sample(parser.experienced_files, math.floor(0.2 * len(parser.experienced_files))))
experienced_session_sample = parser.parse_commands_into_subsets_sliding_window(experienced_session_sample, subset_size)
experienced_session_sample = parser.replace_args_nested(experienced_session_sample)

non_session_sample = parser.parse_commands_per_session(sample(parser.non_programmers_files, math.floor(0.2 * len(parser.non_programmers_files))))
non_session_sample = parser.parse_commands_into_subsets_sliding_window(non_session_sample, subset_size)
non_session_sample = parser.replace_args_nested(non_session_sample)

novice_session_sample = parser.parse_commands_per_session(sample(parser.novice_files, math.floor(0.2 * len(parser.novice_files))))
novice_session_sample = parser.parse_commands_into_subsets_sliding_window(novice_session_sample, subset_size)
novice_session_sample = parser.replace_args_nested(novice_session_sample)

100%|██████████| 11/11 [00:01<00:00,  8.96it/s]


Accuracy predicting N+1 command from N commands with subset of data

In [112]:
from tqdm import tqdm

for i in tqdm(range(1, 5)):
    print("Accuracy Predicting Command {} from commands {} through {}:\n".format(i+1, 1, i))

    print("Scientist session sample:\n", get_accuracy(science_session_sample, graph_depth=i))
    print("Experienced session sample:\n", get_accuracy(experienced_session_sample, graph_depth=i))
    print("Non programmer session sample:\n", get_accuracy(non_session_sample, graph_depth=i))
    print("Novice session sample:\n", get_accuracy(novice_session_sample, graph_depth=i))

  0%|          | 0/4 [00:00<?, ?it/s]

Accuracy Predicting Command 2 from commands 1 through 1:

Scientist session sample:
 Correct Proportion: 68.81% |
 Correct in 15 not 5: 82.54% |
 Has Prediction and is Correct: 68.81% |
 Incorrect Proportion: 31.19% |
 None Proportion: 0.00% |
 First Prediction: 44.93%
Experienced session sample:
 Correct Proportion: 70.46% |
 Correct in 15 not 5: 82.42% |
 Has Prediction and is Correct: 70.46% |
 Incorrect Proportion: 29.54% |
 None Proportion: 0.00% |
 First Prediction: 44.51%
Non programmer session sample:
 Correct Proportion: 72.88% |
 Correct in 15 not 5: 84.91% |
 Has Prediction and is Correct: 72.88% |
 Incorrect Proportion: 27.12% |
 None Proportion: 0.00% |
 First Prediction: 46.52%


 25%|██▌       | 1/4 [14:46<44:18, 886.24s/it]

Novice session sample:
 Correct Proportion: 82.73% |
 Correct in 15 not 5: 92.47% |
 Has Prediction and is Correct: 82.73% |
 Incorrect Proportion: 17.27% |
 None Proportion: 0.00% |
 First Prediction: 51.90%
Accuracy Predicting Command 3 from commands 1 through 2:

Scientist session sample:
 Correct Proportion: 84.20% |
 Correct in 15 not 5: 93.33% |
 Has Prediction and is Correct: 84.21% |
 Incorrect Proportion: 15.79% |
 None Proportion: 0.01% |
 First Prediction: 60.52%
Experienced session sample:
 Correct Proportion: 83.73% |
 Correct in 15 not 5: 92.60% |
 Has Prediction and is Correct: 83.74% |
 Incorrect Proportion: 16.26% |
 None Proportion: 0.01% |
 First Prediction: 61.59%
Non programmer session sample:
 Correct Proportion: 86.02% |
 Correct in 15 not 5: 93.32% |
 Has Prediction and is Correct: 86.02% |
 Incorrect Proportion: 13.98% |
 None Proportion: 0.00% |
 First Prediction: 65.58%


 50%|█████     | 2/4 [15:18<12:47, 383.78s/it]

Novice session sample:
 Correct Proportion: 89.89% |
 Correct in 15 not 5: 96.25% |
 Has Prediction and is Correct: 89.89% |
 Incorrect Proportion: 10.11% |
 None Proportion: 0.00% |
 First Prediction: 66.82%
Accuracy Predicting Command 4 from commands 1 through 3:

Scientist session sample:
 Correct Proportion: 92.72% |
 Correct in 15 not 5: 97.28% |
 Has Prediction and is Correct: 92.72% |
 Incorrect Proportion: 7.28% |
 None Proportion: 0.01% |
 First Prediction: 74.85%
Experienced session sample:
 Correct Proportion: 92.54% |
 Correct in 15 not 5: 96.33% |
 Has Prediction and is Correct: 92.54% |
 Incorrect Proportion: 7.46% |
 None Proportion: 0.00% |
 First Prediction: 76.20%
Non programmer session sample:
 Correct Proportion: 93.66% |
 Correct in 15 not 5: 97.47% |
 Has Prediction and is Correct: 93.66% |
 Incorrect Proportion: 6.34% |
 None Proportion: 0.00% |
 First Prediction: 78.04%


 75%|███████▌  | 3/4 [15:23<03:31, 211.14s/it]

Novice session sample:
 Correct Proportion: 95.06% |
 Correct in 15 not 5: 98.90% |
 Has Prediction and is Correct: 95.06% |
 Incorrect Proportion: 4.94% |
 None Proportion: 0.00% |
 First Prediction: 73.70%
Accuracy Predicting Command 5 from commands 1 through 4:

Scientist session sample:
 Correct Proportion: 81.89% |
 Correct in 15 not 5: 84.08% |
 Has Prediction and is Correct: 82.17% |
 Incorrect Proportion: 17.77% |
 None Proportion: 0.34% |
 First Prediction: 69.53%
Experienced session sample:
 Correct Proportion: 81.36% |
 Correct in 15 not 5: 83.26% |
 Has Prediction and is Correct: 81.59% |
 Incorrect Proportion: 18.35% |
 None Proportion: 0.29% |
 First Prediction: 71.52%
Non programmer session sample:
 Correct Proportion: 85.71% |
 Correct in 15 not 5: 87.88% |
 Has Prediction and is Correct: 85.90% |
 Incorrect Proportion: 14.07% |
 None Proportion: 0.22% |
 First Prediction: 74.03%


100%|██████████| 4/4 [15:25<00:00, 231.35s/it]

Novice session sample:
 Correct Proportion: 90.95% |
 Correct in 15 not 5: 92.81% |
 Has Prediction and is Correct: 91.05% |
 Incorrect Proportion: 8.94% |
 None Proportion: 0.11% |
 First Prediction: 73.83%
